In [2]:
import pandas as pd

product_df_file_path = "./cow_file/original_data.xlsx"
alwaysit_df_file_path = "./cow_file/alwaysIt.xlsx"
eleven_df_file_path = "./cow_file/11st.xls"
naver_df_file_path = "./cow_file/naver2.xlsx"
coupang_df_file_path = "./cow_file/coupang.xlsx"
gmarket_df_file_path = "./cow_file/gmarket.xlsx"

# Load the Excel file into a pandas DataFrame, specifically reading the '상품' sheet
product_df = pd.read_excel(product_df_file_path, sheet_name='상품')
alwaysit_df = pd.read_excel(alwaysit_df_file_path)
eleven_df = pd.read_excel(eleven_df_file_path)
naver_df = pd.read_excel(naver_df_file_path)
coupang_df = pd.read_excel(coupang_df_file_path)
gmarket_df = pd.read_excel(gmarket_df_file_path)


In [2]:
import pandas as pd
import uuid

# Helper function to generate a unique identifier
def generate_uuid():
    return str(uuid.uuid4())

# ETL Function for AlwaysIt
def etl_alwaysit(alwaysit_data, product_df):
    # Initialize lists to store transformed data
    customers = []
    orders = []
    order_details = []
    shipping = []
    
    # Create product id mapping
    product_id_map = {}
    for _, product in product_df.iterrows():
        # Create composite key from name and platform code
        key = f"{product['상품명']}_{product['플랫폼 상품코드']}"
        product_id_map[key] = product['상품 id']
    
    # First create unique customer records
    unique_customers = alwaysit_data[["수령인", "주소"]].drop_duplicates()
    customer_id_map = {}  # To store customer_id mapping
    
    for _, row in unique_customers.iterrows():
        customer_id = generate_uuid()
        customer_id_map[f"{row['수령인']}_{row['주소']}"] = customer_id
        
        customers.append({
            "고객 id": customer_id,
            "고객 전화번호": None,
            "고객 이름": row["수령인"],
            "고객 주소": row["주소"],
            "거래쳐 대상 여부": None
        })

    # Then process orders and related data
    for _, row in alwaysit_data.iterrows():
        # Generate keys
        order_id = generate_uuid()
        shipping_id = generate_uuid()
        order_detail_id = generate_uuid()
        
        # Get customer_id from mapping
        customer_id = customer_id_map[f"{row['수령인']}_{row['주소']}"]

        # Get product_id from mapping
        product_key = f"{row['상품명']}_{row['판매자 상품코드']}"
        product_id = product_id_map.get(product_key, generate_uuid())  # Fallback to new UUID if not found

        # Create order record
        orders.append({
            "주문 id": order_id,
            "상품 id": product_id,
            "고객 id": customer_id,
            "배송 id": shipping_id,
            "플랫폼 수수료 id": None,
            "총 결재액": row["상품가격"] + row["배송비"],
            "총 판매액": row["상품가격"],
            "상품 코드": row["판매자 상품코드"],
            "상품 주문번호": None,
            "주문번호": row["주문아이디"],
            "주문 총 무게": None,
            "주문 날짜": row["주문 시점"],
            "메시지 전송 여부": None,
            "구매자 이름": row["수령인"],
            "주문 상태": None
        })

        # Create order detail record
        order_details.append({
            "주문 상세 id": order_detail_id,
            "상품 id": product_id,
            "주문 id": order_id,
            "수량": row["수량"]
        })

        # Create shipping record
        shipping.append({
            "배송 id": shipping_id,
            "배송사 id": generate_uuid(),
            "주문 id": order_id,
            "주문 상세 id": order_detail_id,
            "배송 메시지": None,
            "배송 날짜": None,
            "해당 배송회차": None,
            "방문수령 여부": None,
            "방문수령 날짜": None,
            "수취자 연락처": row["수령인 연락처"],
            "수취자 휴대폰": None,
            "수취자 전화": None,
            "수최자 주소": row["주소"],
            "수취자 이름": row["수령인"],
            "선착불 여부": None,
            "선착불 금액": None,
            "택배운임 여부": None
        })

    return pd.DataFrame(orders), pd.DataFrame(order_details), pd.DataFrame(shipping), pd.DataFrame(customers)

# Repeat similar logic for 네이버...

In [3]:
# ETL Function for 11번가
def etl_eleven(eleven_data):
    orders = []
    order_details = []
    shipping = []
    customers = []

    for _, row in eleven_data.iterrows():
        # Generate keys
        order_id = generate_uuid()
        customer_id = generate_uuid()
        shipping_id = generate_uuid()
        order_detail_id = generate_uuid()

        # 주문 Table
        orders.append({
            "주문 id": order_id,
            "상품 id": generate_uuid(),
            "고객 id": customer_id,
            "배송 id": shipping_id,
            "플랫폼 수수료 id": None,
            "총 결재액": row["주문금액"],
            "총 판매액": row["판매단가"],
            "상품 코드": row["판매자 상품코드"],
            "상품 주문번호": None,
            "주문번호": row["주문번호"],
            "주문 총 무게": None,
            "주문 날짜": row["결제일시"],
            "메시지 전송 여부": None,
            "구매자 이름": row["구매자"],
            "주문 상태": row["주문상태"]
        })

        # 주문상세 Table
        order_details.append({
            "주문 상세 id": order_detail_id,
            "상품 id": generate_uuid(),
            "주문 id": order_id,
            "수량": row["수량"]
        })

        # 배송 Table
        shipping.append({
            "배송 id": shipping_id,
            "배송사 id": generate_uuid(),
            "주문 id": order_id,
            "주문 상세 id": order_detail_id,
            "배송 메시지": row["배송메시지"],
            "배송 날짜": row["발송처리일"],
            "해당 배송회차": None,
            "방문수령 여부": None,
            "방문수령 날짜": None,
            "수취자 연락처": None,
            "수취자 휴대폰": row["휴대폰번호"],
            "수취자 전화": row["전화번호"],
            "수최자 주소": row["주소"],
            "수취자 이름": row["수취인"],
            "선착불 여부": None,
            "선착불 금액": None,
            "택배운임 여부": None
        })

        # 고객 Table
        customers.append({
            "고객 id": customer_id,
            "고객 전화번호": row["전화번호"],
            "고객 이름": row["구매자"],
            "고객 주소": row["주소"],
            "거래쳐 대상 여부": None
        })

    return pd.DataFrame(orders), pd.DataFrame(order_details), pd.DataFrame(shipping), pd.DataFrame(customers)

In [4]:
def etl_naver(naver_data):
    orders = []
    order_details = []
    shipping = []
    customers = []

    for _, row in naver_data.iterrows():
        # Generate keys
        order_id = generate_uuid()
        customer_id = generate_uuid()
        shipping_id = generate_uuid()
        order_detail_id = generate_uuid()

        # 주문 Table
        orders.append({
            "주문 id": order_id,
            "상품 id": generate_uuid(),
            "고객 id": customer_id,
            "배송 id": shipping_id,
            "플랫폼 수수료 id": None,
            "총 결재액": row["최종 상품별 총 주문금액"],
            "총 판매액": row["상품가격"],
            "상품 코드": row["판매자 상품코드"],
            "상품 주문번호": row["상품주문번호"],
            "주문번호": row["주문번호"],
            "주문 총 무게": None,
            "주문 날짜": row["결제일"],
            "메시지 전송 여부": None,
            "구매자 이름": row["구매자명"],
            "주문 상태": row["주문상태"]
        })

        # 주문상세 Table
        order_details.append({
            "주문 상세 id": order_detail_id,
            "상품 id": generate_uuid(),
            "주문 id": order_id,
            "수량": row["수량"]
        })

        # 배송 Table
        shipping.append({
            "배송 id": shipping_id,
            "배송사 id": generate_uuid(),
            "주문 id": order_id,
            "주문 상세 id": order_detail_id,
            "배송 메시지": row["배송메세지"],
            "배송 날짜": row["발송일"],
            "해당 배송회차": None,
            "방문수령 여부": None,
            "방문수령 날짜": None,
            "수취자 연락처": row["수취인연락처1"],
            "수취자 휴대폰": None,
            "수취자 전화": None,
            "수최자 주소": row["통합배송지"],
            "수취자 이름": row["수취인명"],
            "선착불 여부": None,
            "선착불 금액": None,
            "택배운임 여부": None
        })

        # 고객 Table
        customers.append({
            "고객 id": customer_id,
            "고객 전화번호": row["구매자연락처"],
            "고객 이름": row["구매자명"],
            "고객 주소": row["통합배송지"],
            "거래쳐 대상 여부": None
        })

    return pd.DataFrame(orders), pd.DataFrame(order_details), pd.DataFrame(shipping), pd.DataFrame(customers)

In [5]:
import pandas as pd
import uuid

# Helper function to generate unique identifiers
def generate_uuid():
    return str(uuid.uuid4())

# Function to transform raw data into backend schema
def transform_raw_data(raw_data):
    # Initialize lists for each table
    orders = []
    order_details = []
    shipping = []
    customers = []

    for _, row in raw_data.iterrows():
        # Generate unique IDs for keys
        order_id = generate_uuid()
        customer_id = generate_uuid()
        shipping_id = generate_uuid()
        order_detail_id = generate_uuid()

        # 주문 Table
        orders.append({
            "주문 id": order_id,
            "상품 id": generate_uuid(),  # Generate unique product ID
            "고객 id": customer_id,
            "배송 id": shipping_id,
            "플랫폼 수수료 id": None,  # Not available in raw data
            "총 결재액": row["결제액"],
            "총 판매액": row["옵션판매가(판매단가)"] * row["구매수(수량)"],
            "상품 코드": row["업체상품코드"],
            "상품 주문번호": row["주문번호"],
            "주문번호": row["주문번호"],
            "주문 총 무게": None,  # Not available in raw data
            "주문 날짜": row["주문일"],
            "메시지 전송 여부": None,  # Not available in raw data
            "구매자 이름": row["구매자"],
            "주문 상태": None  # Not available in raw data
        })

        # 주문상세 Table
        order_details.append({
            "주문 상세 id": order_detail_id,
            "상품 id": generate_uuid(),  # Generate unique product ID
            "주문 id": order_id,
            "수량": row["구매수(수량)"]
        })

        # 배송 Table
        shipping.append({
            "배송 id": shipping_id,
            "배송사 id": row["택배사"],
            "주문 id": order_id,
            "주문 상세 id": order_detail_id,
            "배송 메시지": row["배송메세지"],
            "배송 날짜": row["출고일(발송일)"],
            "해당 배송회차": None,  # Not available in raw data
            "방문수령 여부": None,  # Not available in raw data
            "방문수령 날짜": None,  # Not available in raw data
            "수취자 연락처": row["수취인전화번호"],
            "수취자 휴대폰": row["통관용수취인전화번호"],
            "수취자 전화": None,  # Not available in raw data
            "수최자 주소": row["수취인 주소"],
            "수취자 이름": row["수취인이름"],
            "선착불 여부": None,  # Not available in raw data
            "선착불 금액": None,  # Not available in raw data
            "택배운임 여부": None  # Not available in raw data
        })

        # 고객 Table
        customers.append({
            "고객 id": customer_id,
            "고객 전화번호": row["구매자전화번호"],
            "고객 이름": row["구매자"],
            "고객 주소": row["수취인 주소"],
            "거래쳐 대상 여부": None  # Not available in raw data
        })

    # Convert lists to DataFrames
    orders_df = pd.DataFrame(orders)
    order_details_df = pd.DataFrame(order_details)
    shipping_df = pd.DataFrame(shipping)
    customers_df = pd.DataFrame(customers)

    return {
        "orders": orders_df,
        "order_details": order_details_df,
        "shipping": shipping_df,
        "customers": customers_df
    }

# Load the raw data (replace with the actual file path)
x

NameError: name 'x' is not defined